In [7]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETsARD\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')

import PETsARD

In [8]:
# loader = PETsARD.Loader(
#     filepath='../[sunset]/data/[Adt Income] adult.csv',
#     na_values={
#         k: '?' for k in [
#             'workclass',
#             'occupation',
#             'native-country'
#         ]
#     }
# )
# print(loader.data.head(1))


In [9]:
# import boto3
# s3 = boto3.client('s3')
# for bucket in s3.list_buckets()["Buckets"]:
#     print(bucket["Name"])

In [10]:
# import hashlib


# def calculate_sha256(file_path):
#     sha256_hash = hashlib.sha256()
#     with open(file_path, "rb") as f:
#         for byte_block in iter(lambda: f.read(4096), b""):
#             sha256_hash.update(byte_block)
#     return sha256_hash.hexdigest()

# calculate_sha256('benchmark/adult_private_for_demo.csv')

In [11]:
loader = PETsARD.Loader(
    filepath='benchmark://adult',
)
print(loader.data.head(1))

Loader - Benchmarker : Success download the benchmark dataset in https://petsard-benchmark.s3.amazonaws.com/adult.csv.


In [ ]:
loader = PETsARD.Loader(
    filepath='benchmark://adult-private',
)
print(loader.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
